for fp16 raw 
[2024-10-13 16:41:53 root](main_block_ap.py 39): INFO wikitext2 perplexity: 5.47
[2024-10-13 16:41:53 root](main_block_ap.py 39): INFO c4 perplexity: 6.97

for 4bit EfficientQAT 
[2024-10-09 20:37:21 root](main_block_ap.py 39): INFO wikitext2 perplexity: 5.59
[2024-10-09 20:37:21 root](main_block_ap.py 39): INFO c4 perplexity: 7.08
for 4bit EfficientQAT+crossBlock 
[2024-10-10 04:22:32 root](main_block_ap.py 39): INFO wikitext2 perplexity: 5.65
[2024-10-10 04:22:32 root](main_block_ap.py 39): INFO c4 perplexity: 7.12
+epoch: 2 to 5
[2024-10-11 10:11:24 root](main_block_ap.py 39): INFO wikitext2 perplexity: 5.64
[2024-10-11 10:11:24 root](main_block_ap.py 39): INFO c4 perplexity: 7.11
+mse max abs loss
[2024-10-14 17:19:46 root](main_block_ap.py 49): INFO wikitext2 perplexity: 5.98
[2024-10-14 17:19:46 root](main_block_ap.py 49): INFO c4 perplexity: 7.17



2bit learnin rate : 1e-5
EfficientQAT + crossBlock +Magnitude-aware Differentiation learning rate : 1e-5
[2024-10-13 13:56:32 root](main_block_ap.py 39): INFO wikitext2 perplexity: 10.36
[2024-10-13 13:56:32 root](main_block_ap.py 39): INFO c4 perplexity: 11.82


EfficientQAT + crossBlock learning rate : 1e-5
[2024-10-12 07:42:20 root](main_block_ap.py 39): INFO wikitext2 perplexity: 8.60
[2024-10-12 07:42:20 root](main_block_ap.py 39): INFO c4 perplexity: 10.42
add aggressive learning rate 1e-4
[2024-10-12 22:13:54 root](main_block_ap.py 39): INFO wikitext2 perplexity: 13.25
[2024-10-12 22:13:54 root](main_block_ap.py 39): INFO c4 perplexity: 12.70


EfficientQAT

wikitext2:8.45048999786377


In [1]:
from datasets import load_dataset
# 指定缓存目录
cache_dir = '/home/ubuntu/data/exp/dataset'

print("get_wikitext2")
traindata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train',
         cache_dir=cache_dir)
testdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test',
         cache_dir=cache_dir)

traindata = load_dataset(
    'allenai/c4', 
    data_files={'train': 'en/c4-train.00000-of-01024.json.gz'}, 
    split='train', 
    cache_dir=cache_dir
)
valdata = load_dataset(
    'allenai/c4', 
    data_files={'validation': 'en/c4-validation.00000-of-00008.json.gz'}, 
    split='validation', 
    cache_dir=cache_dir
)

traindata = load_dataset("togethercomputer/RedPajama-Data-1T-Sample", split='train',
         cache_dir=cache_dir)


/home/ubuntu/miniconda3/envs/quant/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


get_wikitext2


aqlm quant_model/aqlm/n1024cb1ing8

hf (pretrained=/home/ubuntu/data/exp/model/llama-7b,dtype=float16,parallelize=True), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: 64
|    Tasks    |Version|Filter|n-shot| Metric |Value |   |Stderr|
|-------------|-------|------|-----:|--------|-----:|---|-----:|
|arc_challenge|Yaml   |none  |     0|acc     |0.3763|±  |0.0142|
|             |       |none  |     0|acc_norm|0.3840|±  |0.0142|
|arc_easy     |Yaml   |none  |     0|acc     |0.7134|±  |0.0093|
|             |       |none  |     0|acc_norm|0.6688|±  |0.0097|
|hellaswag    |Yaml   |none  |     0|acc     |0.5123|±  |0.0050|
|             |       |none  |     0|acc_norm|0.6918|±  |0.0046|
|piqa         |Yaml   |none  |     0|acc     |0.7486|±  |0.0101|
|             |       |none  |     0|acc_norm|0.7568|±  |0.0100|
|winogrande   |Yaml   |none  |     0|acc     |0.6535|±  |0.0134|

efficientQAT 2bitg128


|    Tasks    |Version|Filter|n-shot| Metric |Value |   |Stderr|
|-------------|------:|------|-----:|--------|-----:|---|-----:|
|winogrande   |      1|none  |     0|acc     |0.6401|±  |0.0135|
|hellaswag    |      1|none  |     0|acc     |0.4852|±  |0.0050|
|             |       |none  |     0|acc_norm|0.6430|±  |0.0048|
|arc_challenge|      1|none  |     0|acc     |0.3404|±  |0.0138|
|             |       |none  |     0|acc_norm|0.3567|±  |0.0140|
|arc_easy     |      1|none  |     0|acc     |0.7016|±  |0.0094|
|             |       |none  |     0|acc_norm|0.6431|±  |0.0098|
|piqa         |      1|none  |     0|acc     |0.7334|±  |0.0103|
|             |       |none  |     0|acc_norm|0.7416|±  |0.0102|


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

# 简单的线性回归模型
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(1, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 1)
        )

    def forward(self, x):
        return self.linear(x)

# 生成一些虚拟数据
torch.manual_seed(42)
x_train = torch.randn(1000, 1)
y_train = 3 * x_train + 2 + 0.1 * torch.randn(1000, 1)
dev = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# 定义训练函数
def train_model(optimizer, model, epochs=100):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()

        # 清零梯度
        optimizer.zero_grad()

        # 前向传播
        outputs = model(x_train.to(dev))
        loss = criterion(outputs.to(dev), y_train.to(dev))

        # 反向传播和优化
        loss.backward()
        optimizer.step()
    return loss.item()

# 训练和计时函数
def measure_time(foreach, fuse):
    # 定义模型和优化器
    model = LinearRegressionModel()
    model.to(dev)
    optimizer = optim.AdamW(model.parameters(), lr=1e-2, foreach=foreach, fused=fuse)
    
    # 计时
    start_time = time.time()
    final_loss = train_model(optimizer, model)
    end_time = time.time()

    elapsed_time = end_time - start_time
    print(f"foreach={foreach}, fuse={fuse} - Final Loss: {final_loss:.4f}, Time: {elapsed_time:.4f} seconds")

# 运行对比实验
print("Starting timing comparison...\n")
measure_time(foreach=False, fuse=False)  # 不使用优化
measure_time(foreach=True, fuse=False)   # 使用 foreach
measure_time(foreach=False, fuse=True)   # 使用 fuse


Starting timing comparison...

foreach=False, fuse=False - Final Loss: 0.0218, Time: 1.0977 seconds
foreach=True, fuse=False - Final Loss: 0.0137, Time: 1.0349 seconds
foreach=False, fuse=True - Final Loss: 0.0245, Time: 0.8808 seconds
